# Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import warnings
import math
import matplotlib.ticker as ticker
warnings.filterwarnings('ignore') #ignora os avisos


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import tree 

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import plot_roc_curve

# Importação dos dados

In [ ]:
url = 'https://raw.githubusercontent.com/renanmath/Bootcamp_Projeto_Final/main/Dados/dados_resumidos_UTI'

In [ ]:
dados_resumidos = pd.read_csv(url)
dados_resumidos.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,...,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,WINDOW,ICU
0,0,1,0.6,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.183673,0.183673,0.183673,0.183673,-0.891993,-0.891993,-0.891993,-0.891993,0.090147,0.090147,0.090147,0.090147,0.109756,0.109756,0.109756,0.109756,1.000000,1.000000,1.000000,1.000000,-0.835844,-0.835844,-0.835844,...,-0.518519,-0.518519,-0.518519,0.345679,0.345679,0.345679,0.345679,-0.028571,-0.028571,-0.028571,-0.028571,-0.836145,-0.836145,-0.836145,-0.836145,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,0-2,1
2,2,0,0.1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.780261,-0.780261,-0.780261,-0.780261,0.144654,0.144654,0.144654,0.144654,0.158537,0.158537,0.158537,0.158537,1.000000,1.000000,1.000000,1.000000,-0.382773,-0.382773,-0.382773,...,-0.703704,-0.703704,-0.703704,0.345679,0.345679,0.345679,0.345679,0.085714,0.085714,0.085714,0.085714,-0.836145,-0.836145,-0.836145,-0.836145,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,0-2,1
3,3,0,0.4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.326531,0.326531,0.326531,0.326531,-0.891993,-0.891993,-0.891993,-0.891993,-0.203354,-0.203354,-0.203354,-0.203354,-0.219512,-0.219512,-0.219512,-0.219512,-0.828421,-0.828421,-0.828421,-0.828421,-0.729239,-0.729239,-0.729239,...,-0.777778,-0.777778,-0.777778,0.580247,0.580247,0.580247,0.580247,0.200000,0.200000,0.200000,0.200000,-0.937349,-0.937349,-0.937349,-0.937349,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,0-2,0
4,4,0,0.1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.851024,-0.851024,-0.851024,-0.851024,0.358491,0.358491,0.358491,0.358491,0.304878,0.304878,0.304878,0.304878,1.000000,1.000000,1.000000,1.000000,-0.702202,-0.702202,-0.702202,...,-0.592593,-0.592593,-0.592593,0.345679,0.345679,0.345679,0.345679,0.142857,0.142857,0.142857,0.142857,-0.903614,-0.903614,-0.903614,-0.903614,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,-0.076923,-0.351351

In [ ]:
dados_limpos = dados_resumidos.drop(['PATIENT_VISIT_IDENTIFIER', 'WINDOW', 'AGE_ABOVE65', 'GENDER'], axis = 1)
dados_limpos.head()

,AGE_PERCENTIL,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,CALCIUM_MEDIAN,CALCIUM_MEAN,CALCIUM_MIN,CALCIUM_MAX,GLUCOSE_MEDIAN,GLUCOSE_MEAN,GLUCOSE_MIN,GLUCOSE_MAX,HEMATOCRITE_MEDIAN,HEMATOCRITE_MEAN,HEMATOCRITE_MIN,HEMATOCRITE_MAX,HEMOGLOBIN_MEDIAN,HEMOGLOBIN_MEAN,HEMOGLOBIN_MIN,HEMOGLOBIN_MAX,LACTATE_MEDIAN,LACTATE_MEAN,LACTATE_MIN,LACTATE_MAX,LEUKOCYTES_MEDIAN,LEUKOCYTES_MEAN,LEUKOCYTES_MIN,LEUKOCYTES_MAX,LINFOCITOS_MEDIAN,LINFOCITOS_MEAN,...,POTASSIUM_MEDIAN,POTASSIUM_MEAN,POTASSIUM_MIN,POTASSIUM_MAX,SAT02_VENOUS_MEDIAN,SAT02_VENOUS_MEAN,SAT02_VENOUS_MIN,SAT02_VENOUS_MAX,SODIUM_MEDIAN,SODIUM_MEAN,SODIUM_MIN,SODIUM_MAX,UREA_MEDIAN,UREA_MEAN,UREA_MIN,UREA_MAX,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,ICU
0,0.6,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.183673,0.183673,0.183673,0.183673,-0.891993,-0.891993,-0.891993,-0.891993,0.090147,0.090147,0.090147,0.090147,0.109756,0.109756,0.109756,0.109756,1.000000,1.000000,1.000000,1.000000,-0.835844,-0.835844,-0.835844,-0.835844,-0.914938,-0.914938,...,-0.518519,-0.518519,-0.518519,-0.518519,0.345679,0.345679,0.345679,0.345679,-0.028571,-0.028571,-0.028571,-0.028571,-0.836145,-0.836145,-0.836145,-0.836145,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,1
2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.780261,-0.780261,-0.780261,-0.780261,0.144654,0.144654,0.144654,0.144654,0.158537,0.158537,0.158537,0.158537,1.000000,1.000000,1.000000,1.000000,-0.382773,-0.382773,-0.382773,-0.382773,-0.908714,-0.908714,...,-0.703704,-0.703704,-0.703704,-0.703704,0.345679,0.345679,0.345679,0.345679,0.085714,0.085714,0.085714,0.085714,-0.836145,-0.836145,-0.836145,-0.836145,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,1
3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.326531,0.326531,0.326531,0.326531,-0.891993,-0.891993,-0.891993,-0.891993,-0.203354,-0.203354,-0.203354,-0.203354,-0.219512,-0.219512,-0.219512,-0.219512,-0.828421,-0.828421,-0.828421,-0.828421,-0.729239,-0.729239,-0.729239,-0.729239,-0.836100,-0.836100,...,-0.777778,-0.777778,-0.777778,-0.777778,0.580247,0.580247,0.580247,0.580247,0.200000,0.200000,0.200000,0.200000,-0.937349,-0.937349,-0.937349,-0.937349,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,0
4,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.317073,-0.317073,-0.317073,-0.317073,0.357143,0.357143,0.357143,0.357143,-0.851024,-0.851024,-0.851024,-0.851024,0.358491,0.358491,0.358491,0.358491,0.304878,0.304878,0.304878,0.304878,1.000000,1.000000,1.000000,1.000000,-0.702202,-0.702202,-0.702202,-0.702202,-0.641079,-0.641079,...,-0.592593,-0.592593,-0.592593,-0.592593,0.345679,0.345679,0.345679,0.345679,0.142857,0.142857,0.142857,0.142857,-0.903614,-0.903614,-0.903614,-0.903614,0.333333,-0.153846,0.160377,-0.59322

# Criação de variáveis

In [ ]:
features_continuas = dados_limpos.columns[10:-1].tolist()
features_categoricas = dados_limpos.columns[0:9].tolist()

In [ ]:
X = dados_limpos.drop('ICU', axis = 1)
y = dados_limpos['ICU']

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X,y, test_size = 0.25, stratify = y, random_state = 527435)

#GridSeach

## Gradient Boosting

In [ ]:
modelo_grad = GradientBoostingClassifier(random_state=527435)

In [ ]:
parametros_grad = {'loss' : ['deviance', 'exponential'],
              'learning_rate' : [0.01,0.1,1],
              'criterion' : ['friedman_mse', 'mse', 'mae'],
              'max_depth' : [3, 5, 10, 15, 30, 50, 100],
              'max_features' : [None, 'auto', 'log2'],
    
}

In [ ]:
clf_grad = GridSearchCV(modelo_grad,parametros_grad, scoring= 'roc_auc')

In [ ]:
clf_grad.fit(X_treino,y_treino)

GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_n...
                 

In [ ]:
melhor_modelo_grad = clf_grad.best_estimator_
melhor_modelo_grad

GradientBoostingClassifier(ccp_alpha=0.0, criterion='mse', init=None,
                           learning_rate=0.01, loss='exponential', max_depth=5,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=527435, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
clf_grad.best_params_

{'criterion': 'mse',
 'learning_rate': 0.01,
 'loss': 'exponential',
 'max_depth': 5,
 'max_features': 'log2'}

Melhores parâmetros:

{'criterion': 'mse',
 'learning_rate': 0.01,
 'loss': 'exponential',
 'max_depth': 5,
 'max_features': 'log2'}

In [ ]:
clf_grad.best_score_

0.8144466338259442

In [ ]:
y_pred_grad = clf_grad.predict(X_teste)

In [ ]:
print(classification_report(y_teste,y_pred_grad))

              precision    recall  f1-score   support

           0       0.73      0.74      0.74        47
           1       0.70      0.68      0.69        41

    accuracy                           0.72        88
   macro avg       0.71      0.71      0.71        88
weighted avg       0.72      0.72      0.72        88



In [ ]:
melhor_modelo_grad.fit(X_treino, y_treino)
y_pred_grad = melhor_modelo_grad.predict(X_teste)
acur = melhor_modelo_grad.score(X_teste, y_teste)
print(f"Acurácia = {round(100*acur, 2)}%")

Acurácia = 71.59%


In [ ]:
print(classification_report(y_teste,y_pred_grad))

              precision    recall  f1-score   support

           0       0.73      0.74      0.74        47
           1       0.70      0.68      0.69        41

    accuracy                           0.72        88
   macro avg       0.71      0.71      0.71        88
weighted avg       0.72      0.72      0.72        88



In [ ]:
print(classification_report(y_treino,melhor_modelo_grad.predict(X_treino)))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       142
           1       1.00      0.89      0.94       121

    accuracy                           0.95       263
   macro avg       0.96      0.95      0.95       263
weighted avg       0.95      0.95      0.95       263



In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(modelo_log_reg, X_teste, y_teste, ax=ax)

plt.title('Matriz de Confusão (dados reais/dados preditos)')
plt.ylabel('Valores reais',fontsize=14)
plt.xlabel('Valores preditos',fontsize=14)
plt.show()

teste teste teste

we are the champions 

Problema 1:

Calcule o comprimento da parábola $y = x^2$ entre $(0,0)$ e $(1,1)$. 

Problema 2:

Determine a área de região limitada pelas curvas $y = 2x^2 +10$, $y = 4x +16$, $x = -2$ e $x = 5$.

Problema 3:

Seja $f: \mathbb{R} \longrightarrow \mathbb{R}$ uma função contínua. Sabendo que 
$$
\int_{-5}^1 f(x)dx = 7
$$

calcule 
$$
\int_{0}^2 xf(3x-5)dx
$$

Problema 4:

Defina a função $f: (0, \infty) \longrightarrow \mathbb{R}$ dada por 
$$
f(x) = \int_1^x \frac{du}{u}
$$

(a) Mostre que $f$ é crescente.

(b) Verfique que $f(xy) = f(x) + f(y)$ para quaisquer $x,y \in (0,\infty)$. 